In [4]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.color import rgb2gray, gray2rgb
from skimage.transform import hough_circle
from skimage.feature import peak_local_max, canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

from glob import glob

%matplotlib inline

In [5]:
def write_dumbbells(filename, centers, radii, indices):
    counter = 0
    with open(filename,'w') as f:
        f.write('%d\n' % 2)
        for idx in indices:
            x, y = centers[idx]
            r = radii[idx]
            f.write('%4d %9.4f %9.4f %5d %5d %5d %5d %5d\n' % (counter,y,x,r**2,2*r,2*r,r**2*255,-1))
            counter+=1

In [ ]:
visualize = False
for cam in ('1','2','3','4'):
    db = glob('/Users/alex/Documents/OpenPTV/yosef_test/db/db'+cam+'.?????')
    # print db[0]
    for im in db:
        # print im
        image = io.imread(im)
        image_gray = rgb2gray(image)
        # Load picture and detect edges
        image = img_as_ubyte(image_gray)
        edges = canny(image, sigma=3, low_threshold=10, high_threshold=50)
        # fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 8))
        # ax.imshow(edges)

        

        # Detect two radii
        hough_radii = np.arange(30, 60, 2)
        hough_res = hough_circle(edges, hough_radii)

        centers = []
        accums = []
        radii = []

        for radius, h in zip(hough_radii, hough_res):
            # For each radius, extract two circles
            num_peaks = 2
            peaks = peak_local_max(h, num_peaks=num_peaks)
            centers.extend(peaks)
            accums.extend(h[peaks[:, 0], peaks[:, 1]])
            radii.extend([radius] * num_peaks)

        # select the best pair
        # first take the top selection and then remove all
        # the circles which are too close to it
        accums = np.array(accums)
        id1 = np.argsort(accums)[::-1][0]
        # too clsoe
        idf = np.where(((centers - centers[id1])**2).sum(axis=1) < radii[id1]*2)
        accums[idf] = 0.0
        id2 = np.argsort(accums)[::-1][0]
        
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 8))

        # # Draw the most prominent 5 circles
        if visualize:
            image = gray2rgb(image)
            for idx in [id1,id2]: # np.argsort(accums)[::-1]:
                center_x, center_y = centers[idx]
                radius = radii[idx]
                cx, cy = circle_perimeter(center_y, center_x, radius)
                image[cy, cx] = (220, 20, 20)

            ax.imshow(image, cmap=plt.cm.gray)
        
        # write the results to the _targets file
        indices = [id1,id2]
        write_dumbbells(im+'_targets',centers,radii,indices)